In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install bitsandbytes xformers torch torchvision
!pip install -U unsloth transformers accelerate


INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 54.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 200.2/200.2 MB 186.4 MB/s eta 0:00:0100:01      Successfully uninstalled torch-2.6.0+cu124
  

In [3]:
pip install triton==3.2.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 5.4 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.3.0
    Uninstalling triton-3.3.0:
      Successfully uninstalled triton-3.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.7.0 requires triton==3.3.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.2.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.7.0 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install "unsloth[kaggle] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-58ltwrwy/unsloth_0eecc303869d47758e461f04beea38e0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-58ltwrwy/unsloth_0eecc303869d47758e461f04beea38e0
  Resolved https://github.com/unslothai/unsloth.git to commit 5b9f59600e608c3a2fec426cbd5021098e6013d3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached triton-3.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
Using cached triton-3.3.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (156.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [5]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb
from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-06-22 03:23:51.666859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750562631.837632      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750562631.881968      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
user_secrets = UserSecretsClient()
hugging_face_token = user_secrets.get_secret("HF_TOKEN")
wnd_token = user_secrets.get_secret("wnb")

login(hugging_face_token )

wandb.login(key=wnd_token)
run = wandb.init(
    project = "Fine-tuning-Deepseek-R1-Distill-Llama-8B on Medical COT dataset",
    job_type = "training",
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dileeprali303 (dileeprali303-rajiv-gandhi-university-of-knowledge-techn) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
from kaggle_secrets import UserSecretsClient
import wandb
from huggingface_hub import login


user_secrets = UserSecretsClient()


hugging_face_token = user_secrets.get_secret("HF_TOKEN")
wandb_token = user_secrets.get_secret("wnb")


login(hugging_face_token)


wandb.login(key=wandb_token)


run = wandb.init(
    project = "Fine-tuning-Deepseek-R1-Distill-Llama-8B on Medical COT dataset",
    job_type = "training",
    anonymous="allow"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [8]:
from unsloth import FastLanguageModel

max_seq_length = 2048
dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Deepseek-R1-Distill-Llama-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hugging_face_token,
)

==((====))==  Unsloth 2025.6.4: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 6.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [9]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>{}"""

In [10]:

question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or 
              sneezing but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, 
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""


FastLanguageModel.for_inference(model)  


inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  


outputs = model.generate(
    input_ids=inputs.input_ids, 
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200, 
    use_cache=True, 
)


response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])  


<think>
Okay, so I'm trying to figure out what cystometry would show for this 61-year-old woman. Let me break it down step by step.

First, the patient has a history of involuntary urine loss, especially when she coughs or sneezes. That makes me think of stress urinary incontinence. But she doesn't leak at night, so maybe it's not the worst kind. I remember that stress incontinence is usually due to urethral issues, maybe a weak urethral seal.

She underwent a gynecological exam and Q-tip test. I'm not exactly sure what the Q-tip test entails, but I think it's a way to check the urethral function. Maybe they inserted a catheter and checked for proper closure. If the Q-tip doesn't stay upright, it might indicate that the urethra isn't closing properly, leading to leakage.

Now, thinking about cystometry. Cystometry is a test where they fill the bladder and observe how it holds the urine. It can show things like how much the bladder can hold (residual volume) and how the detrusor muscle

In [11]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. 
Please answer the following medical question. 

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


In [12]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en", split = "train[0:500]", trust_remote_code=True)
dataset

README.md:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

Dataset({
    features: ['Question', 'Complex_CoT', 'Response'],
    num_rows: 500
})

In [13]:
dataset[1]

{'Question': 'A 33-year-old woman is brought to the emergency department 15 minutes after being stabbed in the chest with a screwdriver. Given her vital signs of pulse 110/min, respirations 22/min, and blood pressure 90/65 mm Hg, along with the presence of a 5-cm deep stab wound at the upper border of the 8th rib in the left midaxillary line, which anatomical structure in her chest is most likely to be injured?',
 'Complex_CoT': "Okay, let's figure out what's going on here. A woman comes in with a stab wound from a screwdriver. It's in her chest, upper border of the 8th rib, left side, kind of around the midaxillary line. First thought, that's pretty close to where the lung sits, right?\n\nLet's talk about location first. This spot is along the left side of her body. Above the 8th rib, like that, is where a lot of important stuff lives, like the bottom part of the left lung, possibly the diaphragm too, especially considering how deep the screwdriver went.\n\nThe wound is 5 cm deep. Tha

In [14]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [15]:

def formatting_prompts_func(examples):  
    inputs = examples["Question"]       
    cots = examples["Complex_CoT"]      
    outputs = examples["Response"]      
    
    texts = []  
    
   
    for input, cot, output in zip(inputs, cots, outputs):  
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN  
        texts.append(text)  
    return {
        "text": texts,   
    }

In [16]:
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

"Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are a medical expert with advanced knowledge in clinical reasoning, diagnostics, and treatment planning. \nPlease answer the following medical question. \n\n### Question:\nGiven the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?\n\n### Response:\n<think>\nOkay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg i

In [17]:
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  
    target_modules=[  
        "q_proj",  
        "k_proj",   
        "v_proj",   
        "o_proj",  
        "gate_proj",  
        "up_proj",    
        "down_proj",  
    ],
    lora_alpha=16,  
    lora_dropout=0,  
    bias="none",  
    use_gradient_checkpointing="unsloth", 
    random_state=3407,  
    use_rslora=False,  
    loftq_config=None, 
)

Unsloth 2025.6.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [18]:

trainer = SFTTrainer(
    model=model_lora,  
    tokenizer=tokenizer, 
    train_dataset=dataset_finetune,  
    dataset_text_field="text",  
    max_seq_length=max_seq_length,  
    dataset_num_proc=2, 
   
    args=TrainingArguments(
        per_device_train_batch_size=2,  
        gradient_accumulation_steps=4,  
        num_train_epochs=1, 
        warmup_steps=5,  
        max_steps=60,  
        learning_rate=2e-4,  
        fp16=False, 
        bf16=False, 
        logging_steps=10,  
        optim="adamw_8bit",  
        weight_decay=0.01,  
        lr_scheduler_type="linear",  
        seed=3407,  
        output_dir="outputs",  
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

In [19]:
import os
os.environ['TRITON_JIT_DISABLE_OPT'] = '1'  

trainer = SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=dataset_finetune,
    dataset_text_field="text",
    max_seq_length=1024, 
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=False,  
        bf16=False, 
        logging_steps=10,
        optim="paged_adamw_32bit",  
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        gradient_checkpointing=False, 
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/500 [00:00<?, ? examples/s]

In [20]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.938100
20,1.417800
30,1.397600
40,1.354700
50,1.383800
60,1.363500


In [21]:
# Save the adapter and tokenizer
model_lora.save_pretrained("lora_adapter")
tokenizer.save_pretrained("lora_adapter")


('lora_adapter/tokenizer_config.json',
 'lora_adapter/special_tokens_map.json',
 'lora_adapter/chat_template.jinja',
 'lora_adapter/tokenizer.json')

In [22]:
wandb.finish()

train/epoch,▁▂▄▅▇██
train/global_step,▁▂▄▅▇██
train/grad_norm,█▂▁▁▂▁
train/learning_rate,█▇▅▄▂▁
train/loss,█▂▂▁▁▁
total_flos,1.6770438117310464e+16
train/epoch,0.96
train/global_step,60
train/grad_norm,0.25614
train/learning_rate,0.0
train/loss,1.3635


In [24]:
question = """A 61-year-old woman with a long history of involuntary urine loss during activities like coughing or sneezing 
              but no leakage at night undergoes a gynecological exam and Q-tip test. Based on these findings, 
              what would cystometry most likely reveal about her residual volume and detrusor contractions?"""


FastLanguageModel.for_inference(model_lora) 


inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")


outputs = model_lora.generate(
    input_ids=inputs.input_ids,          
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200,                  
    use_cache=True,                        
)


response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])


<think>
Okay, so let's break this down. We have a 61-year-old woman who's been dealing with involuntary urine loss during things like coughing or sneezing, but she's not having any problems at night. That's interesting because it usually points towards some kind of issue with her bladder or urethra, but not necessarily something that affects her sleep.

When they did the gynecological exam and the Q-tip test, they found something. Now, I'm not entirely sure what they found, but I'm guessing it's not something that's super serious. The Q-tip test is usually done to see if there's a urethral obstruction or something like that. So if it's positive, it suggests there's an obstruction. If it's negative, then maybe it's more of a bladder issue.

Now, the big question is what would cystometry show. Cystometry is like a test where they fill up the bladder and see how it behaves. If there's a residual volume that's more than usual, it could mean that her bladder isn't emptying completely. That

In [25]:
model_lora.save_pretrained("lora_adapter")
tokenizer.save_pretrained("lora_adapter")


('lora_adapter/tokenizer_config.json',
 'lora_adapter/special_tokens_map.json',
 'lora_adapter/chat_template.jinja',
 'lora_adapter/tokenizer.json')

In [26]:
question = """A 59-year-old man presents with a fever, chills, night sweats, and generalized fatigue, 
              and is found to have a 12 mm vegetation on the aortic valve. Blood cultures indicate gram-positive, catalase-negative, 
              gamma-hemolytic cocci in chains that do not grow in a 6.5% NaCl medium. 
              What is the most likely predisposing factor for this patient's condition?"""


FastLanguageModel.for_inference(model_lora) 


inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")


outputs = model_lora.generate(
    input_ids=inputs.input_ids,          
    attention_mask=inputs.attention_mask, 
    max_new_tokens=1200,                  
    use_cache=True,                        
)


response = tokenizer.batch_decode(outputs)


print(response[0].split("### Response:")[1])


<think>
Okay, let's see. This guy is 59 and has a fever, chills, night sweats, and feels really tired. That sounds like a classic picture of endocarditis. It’s a heart infection, and those symptoms are pretty common with it. 

Now, looking at the blood culture results: gram-positive, catalase-negative, gamma-hemolytic cocci in chains. Hmm, those are pretty distinctive. Let's break it down. 

The gram-positive cocci in chains are a big clue. They are the classic picture of Streptococcus. And, wait, they don’t grow in a 6.5% NaCl medium. That’s a key identifier for enterococci, which don’t grow under these conditions. But we’re dealing with Streptococcus here, right?

So, we're talking about Streptococcus, which is gram-positive, but it doesn't fit the criteria for enterococci. Now, let's think about the heart valve vegetation. The vegetation is on the aortic valve, which is a pretty common spot for endocarditis. 

The question is about the predisposing factors. There are a few possibil

In [34]:
with open("app.py", "w") as f:
    f.write(code)
